# Fake news detection

#### Краткое описание работы

Использую предобученные эмбеддинги от navec, далее пробовал сделать простую, незатратную, как оказалось, для своей простоты очень эффективную модель: векторное представление заголовка -> среднее эмбеддингов всех слов в нем

Очевидно, что так связи между словами, контекст, будет плохо учитываться, поэтому лучше использовать рекурентную нейронную сеть, а именно LSTM

Работа с незнакомыми словами:                            
1. для каждого слова из navec.vocab.words получаю n-граммы
2. каждой уникальной n-грамме ставлю в соотвествие среднее всех векторов в которых она содержится
3. если вижу незнакомое слово, то разбиваю его на n-граммы
4. усредняю n-граммы незнакомого слова(те, что встречались в n-граммах от navec)

Для работы с нейронными сетями выбрал pytorch

LSTM + один линейный слой

функция потерь: CrossEntropyLoss

оптимизатор: Adam

#### импорты

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import Adam

import string

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lvg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# !pip install ipywidgets

In [3]:
# !pip install jupyterlab_widgets

#### загрузка данных

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
!pip install navec

In [4]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar

--2022-04-23 16:13:19--  https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26634240 (25M) [application/x-tar]
Saving to: ‘navec_news_v1_1B_250K_300d_100q.tar.2’

navec_news_v1_1B_25 100%[===================>]  25.40M  9.15MB/s    in 2.8s    

2022-04-23 16:13:22 (9.15 MB/s) - ‘navec_news_v1_1B_250K_300d_100q.tar.2’ saved [26634240/26634240]



In [6]:
from navec import Navec

navec_news = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')  

In [7]:
data = pd.read_csv('train.tsv', sep='\t')

In [8]:
data.head()

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1


#### разделим на train и test

In [9]:
# разделим на train и test
train_data, test_data = train_test_split(data, shuffle = True, test_size=0.2, random_state=44)

# обновим идексы
train_data.reset_index(drop = True, inplace = True)
test_data.reset_index(drop = True, inplace = True)
 
#посморим на наши датасеты
print('train:')
print(train_data.describe())
print('test:')
print(test_data.describe())

train:
           is_fake
count  4606.000000
mean      0.499566
std       0.500054
min       0.000000
25%       0.000000
50%       0.000000
75%       1.000000
max       1.000000
test:
           is_fake
count  1152.000000
mean      0.501736
std       0.500214
min       0.000000
25%       0.000000
50%       1.000000
75%       1.000000
max       1.000000


#### чтобы бороться с незнакомыми словами создадим словарь с 4граммами(по типу fasttext):
каждой 4грамме будет ставиться в соответствие среднее векторов от слов в которых встречается эта 4грамма

In [13]:
def get_ngrams(list_of_words, chars, embedding_model):
  ngrams = {}
  for word in list_of_words:
    vector_of_word = embedding_model[word]
    for i in range(len(word)-chars + 1):
      seq = word[i:i+chars]
      if seq in ngrams.keys():
        ngrams[seq].append(vector_of_word)
      else:
        ngrams[seq] = [vector_of_word]
  # усредним
  for seq in ngrams:
    ngrams[seq] = np.array(ngrams[seq]).mean(axis = 0)
  return ngrams

In [14]:
ngrams_of_navec = get_ngrams(list_of_words = navec_news.vocab.words, chars = 4, embedding_model = navec_news)
# ngrams_of_navec.keys()

#### Найдем mean и std для нормализации эмбеддингов, прежде чем обучать на них сеть. 

In [15]:
# из предобученных эмбеддингов
navec_news_vectors = np.array([navec_news[word] for word in navec_news.vocab.words])

# из ngrams
ngrams_vectors = np.array(list(ngrams_of_navec.values()))

emb_vectors = np.concatenate((ngrams_vectors, navec_news_vectors))

emb_mean = np.mean(emb_vectors, 0)
emb_std = np.std(emb_vectors, 0)

#### функция для векторного представления слов в заголовке

In [16]:
# функция, которая получает эмбеддинг заголовка, как список всех эмбеддингов слов в заголовке
def get_seq_emb_of_title(title, tokenizer, chars = 4):
  """
  возвращает эмбеддинг заголовка, как список всех эмбеддингов слов в нем:
  title - заголовок, который нужно представить в векторном виде
  tokenizer - ваш токенизатор
  chars - кол-во символов при разбиении на n-граммы
  """
  # разбиваем на токены, приведя заголовок к нижнему регистру
  tokens = tokenizer.tokenize(title.lower())

  # фильтруем токены по длине, избавляемся от стопслов и чисел
  filtered_tokens = [token for token in tokens if (len(token) > 3) and 
                     token not in stopwords.words('russian') and 
                     token.isalpha()]
  # построим эмбединги
  embeddings = []
  for token_word in filtered_tokens:

    if token_word in navec_news:  # для тех, которые есть в нашей модели
      embeddings.append((navec_news[token_word] - emb_mean) / emb_std) # сразу же нормализуем

    else: # для тех слов, которых нет, строим его n-граммы
      ngrams= []
      # создаем n-граммы для незнокомого слова
      for i in range(len(token_word)-chars + 1):
        ngrams.append(token_word[i:i+chars])
      
      # для каждой n-граммы получаем ее представление из ngrams_of_navec
      for ngram in ngrams:
        if ngram in ngrams_of_navec.keys():
          embeddings.append(ngrams_of_navec[ngram])

  if len(embeddings) == 0:
      return navec_news['<pad>'].reshape(1, 300)
  elif len(embeddings) == 1:
      return np.array(embeddings).reshape(1, 300)
  else:
      return np.array(embeddings).reshape(len(embeddings), 300)

In [17]:
# для каждого заголовка получим его эмбеддинг
embs_train = []
embs_test = []

for i in range(len(train_data['title'])):
  embs_train.append(get_seq_emb_of_title(train_data['title'][i], 
                                                    tokenizer = nltk.WordPunctTokenizer(), chars = 4))

for i in range(len(test_data['title'])):
  embs_test.append(get_seq_emb_of_title(test_data['title'][i], 
                                                   tokenizer = nltk.WordPunctTokenizer(), chars = 4))

# создадим новую колонку, в которой будем хранить эмбеддинги предложений
train_data['emb'] = embs_train
test_data['emb'] = embs_test

In [83]:
print(train_data['emb'].shape)
print(train_data['emb'][4].shape)

(4606,)
(3, 300)


In [19]:
# перемешаем индексы
n_train = len(train_data['emb'])
n_test = len(test_data['emb'])
indexes = np.arange(n_train)
np.random.shuffle(indexes)

# и будем хранить в dict для удобства
train_dataset = {'features': np.array([train_data['emb'][i][0] for i in indexes]),  
                 'targets': np.array([train_data['is_fake'][i] for i in indexes])}

test_dataset = {'features': np.array([test_data['emb'][i][0] for i in range(n_test)]), 
                'targets': np.array([test_data['is_fake'][i] for i in range(n_test)])}

#### функция - батчгенератор

In [20]:
# батч-генератор, который генерирует батчи, внутри каждого батча делаем ПАДдинг до максимальной длины заголовка в батче
def custom_batch_generator(dataset, batch_size, features, targets, shuffle=True):
    """
    нарезает на батчи, дополняя нулевыми векторами до максимальной длины заголовка в батче
    dataset - датасет в формате MyDataset с tensor-ами внутри
    batch_size - размер батча
    features: int - индекс столбца с фичами, по умолчанию 0
    targets: int - индекс столбца с таргетами, по умолчанию 1
    shuffle - перемешивать ли индексы, по умолчанию True
    """
    # X, Y = dataset[:-1], dataset[1:]
    X = dataset[features]
    Y = dataset[targets]
    
    PAD = np.zeros((1, 300))
    n_samples = len(X)

# генерим список индексов
    list_of_indexes = np.linspace(0, n_samples - 1, n_samples, dtype=np.int64)
    List_X = []
    List_Y = []
    
# если нужно перемешать, то перемешиваем
    if shuffle:
        np.random.shuffle(list_of_indexes)
        

# сгенерируем список индексов по этим индексам,
# сделаем новый перемешаный список токенов и тэгов
    for indx in list_of_indexes:
        List_X.append(X[indx])
        List_Y.append(Y[indx])
    
    n_batches = n_samples//batch_size
    if n_samples%batch_size != 0:
        n_batches+=1
    
# для каждого k и пары X и Y
    for k in range(n_batches):
# указываем текущии размер батча
        this_batch_size = batch_size
    
# если мы выдаем последний батч, то его нужно обрезать
        if k == n_batches - 1:
            if n_samples%batch_size > 0:
                this_batch_size = n_samples%batch_size
                
        This_X = List_X[k*batch_size:k*batch_size + this_batch_size]
        This_Y = List_Y[k*batch_size:k*batch_size + this_batch_size]
        
        # This_X_line = [
        #                [word2vec.get(char, 0) for char in sent]\
        #                for sent in This_X]
        # This_Y_line = [
        #                [word2vec.get('<START>', 0)]\
        #                + [word2vec.get(char, 0) for char in sent]\
        #                + [word2vec.get('<FINISH>', 0)]\
        #                for sent in This_Y]

        List_of_length_x = [len(sent) for sent in This_X]
        length_of_sentence_x = max(List_of_length_x)

        # List_of_length_y = [len(sent) for sent in This_Y]
        # length_of_sentence_y = max(List_of_length_y)

        # x_arr = np.ones(shape=[this_batch_size, length_of_sentence_x])*PAD
        x_arr = np.zeros(shape=[this_batch_size, length_of_sentence_x, 300])
        # y_arr = np.ones(shape=[this_batch_size, length_of_sentence_y])*PAD
        y_arr = np.array(This_Y)

        for i in range(this_batch_size):
            x_arr[i, :len(This_X[i])] = This_X[i]
            # y_arr[i, :len(This_Y_line[i])] = This_Y_line[i]

        x = torch.Tensor(x_arr)
        y = torch.LongTensor(y_arr)
        lengths = torch.LongTensor(List_of_length_x)

        yield x, y

#### функции - обучатели

In [21]:
def trainer(count_of_epoch, batch_size, dataset, model, loss_function, optimizer, batch_generator, lr = 0.001):
    """
    итерируемся по кол-ву эпох и вызывает функцию train_epoch
    count_of_epoch - кол-во эпох
    batch_size - размер батча
    dataset - данные для обучения
    model - модель нейронной сети
    loss_function - функция потерь
    optimizer - оптимизатор
    batch_generator - батч-генератор, если используется кастомный
    lr - скорость обучения, по умолчанию 0.001
    """
    optima = optimizer(model.parameters(), lr=lr)
    
    iterations = tqdm(range(count_of_epoch), desc='epoch')
    iterations.set_postfix({'train epoch loss': np.nan})
    for it in iterations:

        number_of_batch = len(dataset)//batch_size + (len(dataset)%batch_size>0)

        generator = tqdm(custom_batch_generator(dataset, batch_size=batch_size, features='features', targets='targets', shuffle=True), 
                         leave=False, total=number_of_batch)

        epoch_loss = train_epoch(train_generator=generator, 
                    model=model, 
                    loss_function=loss_function, 
                    optimizer=optima)
        
        iterations.set_postfix({'train epoch loss': epoch_loss})

In [22]:
def train_epoch(train_generator, model, loss_function, optimizer):
    """
    внутри итерируемся по батчам внутри батчгенератора
    train_generator - батчгенератора
    model - модель нейронной сети
    loss_function - функция потерь
    optimizer - оптимизатор
    """
    epoch_loss = 0
    total = 0
    for it, (batch_of_x, batch_of_y) in enumerate(train_generator):
        batch_loss = train_on_batch(model, batch_of_x, batch_of_y, optimizer, loss_function)
            
        epoch_loss += batch_loss*len(batch_of_x)
        total += len(batch_of_x)
    
    return epoch_loss/total

In [23]:
def train_on_batch(model, x_batch, y_batch, optimizer, loss_function):
    """
    в обучаемся на одном батче
    model - модель нейронной сети
    x_batch - фичи
    y_batch - таргеты(метки классов)
    optimizer - оптимизатор
    loss_function - функция потерь
    """
    model.train()
    optimizer.zero_grad()
    
    output = model(x_batch.to(device))
    
    loss = loss_function(output, y_batch.to(device))
    loss.backward()

    optimizer.step()
    return loss.cpu().item()

#### модель RNN с использованием LSTM

In [84]:
class RNNclassifier(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device
    def __init__(self, output_dim, emb_dim = 300, hidden_dim = 100, 
                 num_layers = 3, bidirectional = False, p=0.7):
        super(RNNclassifier, self).__init__()
        self.lstm = torch.nn.LSTM(emb_dim, hidden_dim, num_layers, 
                                     bidirectional=bidirectional, 
                                     batch_first=True, dropout=p)
        self.linear = torch.nn.Linear(
            2*num_layers*int(bidirectional + 1)*hidden_dim,
            output_dim)
    def forward(self, input):
        _, (h, c) = self.lstm(input)
        act = torch.cat([h, c], dim=0).transpose(0, 1)
        act = act.reshape(len(input), -1)
        return self.linear(act)

#### обучение

инициализация модели

In [85]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

In [98]:
rnn_model = RNNclassifier(output_dim = 2, 
                          emb_dim = 300, 
                          hidden_dim = 10, 
                          num_layers = 3, 
                          bidirectional = False,
                          p = 0.7)

_ = rnn_model.to(device)

качество до обучения

In [99]:
batch_generator = custom_batch_generator(dataset = test_dataset, features = 'features', targets='targets', batch_size = 64)

pred = []
real = []
rnn_model.eval()
for it, (x_batch, y_batch) in enumerate(batch_generator):
    x_batch = x_batch.to(device)
    with torch.no_grad():
        output = rnn_model(x_batch)

    pred.extend(torch.argmax(output, dim=-1).cpu().numpy().tolist())
    real.extend(y_batch.cpu().numpy().tolist())

print(classification_report(real, pred))

              precision    recall  f1-score   support

           0       0.51      0.39      0.44       574
           1       0.51      0.62      0.56       578

    accuracy                           0.51      1152
   macro avg       0.51      0.51      0.50      1152
weighted avg       0.51      0.51      0.50      1152



In [100]:
trainer(count_of_epoch = 50,
        batch_size = 64, 
        dataset = train_dataset,
        model = rnn_model, 
        loss_function = loss_function,
        optimizer = optimizer,
        batch_generator = custom_batch_generator,
        lr = 0.0001)

epoch:   0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [101]:
batch_generator = custom_batch_generator(dataset = test_dataset, features = 'features', targets='targets', batch_size = 64)


pred = []
real = []
rnn_model.eval()
for it, (x_batch, y_batch) in enumerate(batch_generator):
    x_batch = x_batch.to(device)
    with torch.no_grad():
        output = rnn_model(x_batch)

    pred.extend(torch.argmax(output, dim=-1).cpu().numpy().tolist())
    real.extend(y_batch.cpu().numpy().tolist())

    
print(classification_report(real, pred))

              precision    recall  f1-score   support

           0       0.55      0.49      0.52       574
           1       0.54      0.60      0.57       578

    accuracy                           0.55      1152
   macro avg       0.55      0.55      0.54      1152
weighted avg       0.55      0.55      0.54      1152



Вывод: долго обучается, не достигая желаемого качества, поэтому разумнее использовать простую модель
(см. другой ноутбук)